A place to test random stuff

In [3]:
from monkey.model.classification_model.efficientnet_b0 import (
    EfficientNet_B0,
)
import torch
import torch.nn as nn
from torchsummary import summary
from monkey.model.utils import (
    get_classification_metrics,
    get_activation_function,
)
from monkey.model.loss_functions import get_loss_function

In [14]:
model = EfficientNet_B0(input_channels=3, num_classes=1)
model = model.eval()
summary(model, input_size=(3, 32, 32))
print(model)

Layer (type:depth-idx)                             Param #
├─EfficientNet: 1-1                                --
|    └─Conv2d: 2-1                                 864
|    └─BatchNormAct2d: 2-2                         --
|    |    └─Identity: 3-1                          --
|    |    └─SiLU: 3-2                              --
|    └─Sequential: 2-3                             --
|    |    └─Sequential: 3-3                        1,448
|    |    └─Sequential: 3-4                        16,714
|    |    └─Sequential: 3-5                        46,640
|    |    └─Sequential: 3-6                        242,930
|    |    └─Sequential: 3-7                        543,148
|    |    └─Sequential: 3-8                        2,026,348
|    |    └─Sequential: 3-9                        717,232
|    └─Conv2d: 2-4                                 409,600
|    └─BatchNormAct2d: 2-5                         --
|    |    └─Identity: 3-10                         --
|    |    └─SiLU: 3-11                

In [36]:
test = torch.randn(size=(4, 3, 32, 32), dtype=torch.float32)
gt = torch.tensor([0, 0, 0, 1], dtype=torch.float)
gt = torch.unsqueeze(gt, 1)

loss_fn = nn.BCELoss()

pred_labels = []

act = get_activation_function("sigmoid")

with torch.no_grad():
    out = model(test)
    print(out.shape)
    out = act(out)
    # out = torch.squeeze(out)
    pred = (out > 0.5).float()
    pred = torch.squeeze(pred)
    print(pred)
    pred = pred.cpu().tolist()

    pred_labels.extend(pred)

    loss = loss_fn(out, gt)


gt_labels = torch.squeeze(gt).cpu().tolist()

print(f"probs {out}")
print(out.shape)
print(f"gt {gt}")
print(gt.shape)
print(f"loss {loss}")
# print(f"weighted loss {weighted_loss}")

print(f"pred labels: {pred_labels}")
print(f"true labels: {gt_labels}")

metrics = get_classification_metrics(gt_labels, pred_labels)
print(metrics)

torch.Size([4, 1])
tensor([1., 1., 1., 1.])
probs tensor([[0.9614],
        [0.9268],
        [0.9159],
        [0.9137]])
torch.Size([4, 1])
gt tensor([[0.],
        [0.],
        [0.],
        [1.]])
torch.Size([4, 1])
loss 2.109078884124756
pred labels: [1.0, 1.0, 1.0, 1.0]
true labels: [0.0, 0.0, 0.0, 1.0]
{'Balanced_Accuracy': 0.5, 'Precision': 0.25, 'Recall': 1.0, 'F1': 0.4}
